# Intro RL para LLMs

- Agent/policy model: Estrategia para tomar decisiones. Es el LLM
- Environment: provee feedback al agente. Puede ser muchas cosas aca
- Action: Pueden ser PALABRAS, RESPUESTA ENTERA, OUTCOME FINAL, etc
- Reward: Feedback que le da el env al agent. Numero

---

**RLHF**

- Preferencias humanas: a partir de varias respuestas, las ranqueamos
- Reward model: construimos un reward model basado en esas preferencias. Ahora tenemos un modelo que, dado una respuesta, estima el puntaje que le darian humanos
- Fine-tune el LLM con RL: Ahora el reward model nos da el feedback. Entonces:
    - El agente genera respuestas
    - El reward model las scorea
    - Ajustamos los weights para que tienda a producir outputs que sean bien puntuados


---

**RL vs SL**

Principalmente cambia la fuente de la señal (reward). Qué optimizas realmente
- SL: Minimizar divergencia respecto a la distribución objetivo (imitar). Tiene un reward **denso** porque sabes la respuesta target para cada token.
- RL: Maximizar retorno esperado bajo dinámicas del entorno (elegir). La señal es escasa y esta diferida en el tiempo. Se requiere **exploracion** y credito temporal. 

Por eso se dice que SFT memorizes y RL generalizes https://arxiv.org/abs/2501.17161

---



# Policy optimization (gradient-based)

Tecnicas para optimizar el LLM / policy model para que mejore las predicciones.

**Intro**:

En SL tenemos una funcion de perdida (loss) y lo que queremos es minimizar esa loss. Entonces dado un batch, calculamos la loss (un numero final) basado en muchos errores (1 o varios para cada elemento del batch), los promediamos y nos queda ese numero final. Ahi, buscamos el gradiente (el conjunto de derivadas parciales de cada uno de los parametros aprendibles) que hagan que mas baje la loss en ese punto. Y actualizamos los valores de los parametros en la direccion y fuerza de esas derivadas y agregandole otros empujones.

En RL, lo que tratamos de hacer es subir o bajar la probabilidad de los tokens segun si aparecen en la secuencia que genero un buen o mal resultado (bien o mal comparado con un baseline).


**Pasos general (on-policy)**:
- Se hacen rollouts (el modelo genera respuestas completas)
- Se evalua cada respuesta y se obtiene score o reward
- El score se convierte en ventaje (advantage): cuanto mejor o peor esta respuesta fue a lo esperado o tipico (por ej del grupo o de algo de referencia).
    - Si tengo solo reward final, el advantage se aplica a todos los tokens de la secuencia de la misma manera (credito global)
    - Si tengo verificadores por paso, se reparten ventajas distintas por token
- Distribuyo la ventaja para cada token:
    - Si es positiva, subo su log-prob (empujo logits para arriba).
    - Si es negativa, la bajo.
    - Indirectamnete afecto a las otras (porque en probs la suma = 1)
- Regularizo para no irme tan lejos de una distribucion anterior (basada en una politica de referencia) para evitar colapso.

**Caso general**
- Nosotros tenemos una red y para un estado (secuenci), produce logits (next token / action)
- Los pasamos por softmax y tenemos probs y de esos tomamos el log, por lo que trabajamos con log probs. 
- Para el token elegido en un paso t, vemos el log prob (solo tenemos un log prob en ese paso)
- Si tomamos el gradiente de ese log prob, estamos viendo un conjunto de derivadas parciales del logprob con respecto a todos los parametros del modelo. Esto indica la direccion de MAXIMA SUBIDA -> o sea, si tomamos un paso en esa direccion, aumentamos la logprob de ese token para la prox.
- Si hacemos lo mismo para cada token de la secuencia de pasos generados, y sumamos los gradientes (sumamos las derivadas parciales (o sea, para un parametro miramos las derivadas parciales por cada token en la secuencia generada y los sumamos)), nos queda un gradiente unico que apunta a maximizar todos los tokens en la secuencia generada. 
- Despues vemos el reward para esa secuencia y lo comparamos con el baseline (R-b). Si el baseline es 0.5 y nuestro reward es 0.2, el advantage da -0.3. Eso lo usamos para multiplicar, lo que nos cambia la direccion del gradiente si el advantage es negativo y tambien nos da una magnitud de cambio (escala el tamaño del paso junto con el lr). O sea que si es negativo, en vez de apuntar a aumentar la prob, apunta a disminuir la prob de los tokens.
- Para no favorecer secuencias largas, a veces se promedia la suma de logprobs y se suele agregar entropia (para mantener diversidad) y KL a un modelo de referencia para regularizar.


## REINFORCE

Loss function: 

$$\text L_\theta = - (R - b) \sum_t \log \pi_\theta(a_t \mid s_t)$$

- Partimos de Maximum Likelihood: queremos que el modelo asigne alta probabilidad a lo que efectivamente ocurrió (los datos reales).
- Para secuencias, eso se traduce en un producto de probabilidades de cada token de la secuencia.
- Como trabajar con productos es numéricamente inestable (tienden a 0), usamos logaritmo: eso convierte el producto en suma.
- Advantage 𝐴=𝑅−𝑏: En RL, no todas las muestras valen lo mismo: algunas tuvieron mejor reward. Por eso ponderamos la suma con el advantage. El Advantage puede ser por paso (cuando haya reward por paso u otra tecnica) o general para la secuencia (para una secuencia particular, con respecto a otras (baseline)). Se puede poner dentro de la suma.

$$\text L_\theta = - \sum_t A_t \cdot \log \pi_\theta(a_t \mid s_t)$$

- La loss depende de θ. Como regla de gradient descent tenemos: $\theta \leftarrow \theta - a \cdot \nabla_\theta L$.
- Queremos derivar la loss respecto a θ, modificando los pesos (que es lo unico que podemos mover) para que la loss sea mas chica.
- Tenemos la politica (que es el LLM), que esto lo podemos mover, porque depende de θ. Entonces lo queremos derivar: $\nabla_\theta \log \pi_\theta(a_t \mid s_t)$
- El gradiente de una funcion siempre nos dice el steepest ascent de la funcion con respecto a un parametro.
- Si tenemos una Loss, vamos a querer derivarla respecto a los parametros del modelo. Entonces buscamos en la formula aquello que dependa de los parametros del modelo (pi)

Entonces, el gradiente para la secuencia entera seria:

$$\nabla_\theta L_\theta = - \sum_t A_t \cdot \nabla_\theta \log \pi_\theta(a_t \mid s_t)$$

Este es el vector del gradiente, que apunta a donde mas crece L.

Y en cuanto a gradient descent tenemos que, en cada optimizacion, theta cambia asi:

$$\Delta \theta = - a \cdot \nabla_\theta L(\theta)$$

O sea: 

$$\theta \leftarrow \theta - a \cdot \nabla_\theta L(\theta)$$

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import math

torch.manual_seed(0)

vocab_size=5
T=3
lr=0.5

# Model: linear map from one-hot(state) to vocab logits (bias-free for clarity).
model = nn.Linear(T, vocab_size, bias=False)
# Initialize weights small & reproducible
with torch.no_grad():
    model.weight.copy_(0.1 * torch.randn(vocab_size, T))

opt = optim.SGD(model.parameters(), lr=lr)

log_softmax = nn.LogSoftmax(dim=-1)

model.weight

Parameter containing:
tensor([[ 0.0604,  0.0811, -0.0045],
        [ 0.0880,  0.1048, -0.0045],
        [-0.0723,  0.2866, -0.0566],
        [ 0.0160, -0.0025,  0.1074],
        [ 0.2263, -0.0918, -0.0225]], requires_grad=True)

In [31]:
# Inputs: one-hot vectors for each time step 0..T-1
I = torch.eye(T)  # shape (T, T)
I

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])

In [32]:
# CASO 1 (GOOD)
chosen_tokens = [2, 1, 4]
advantage = 0.4

# --- Forward BEFORE update: record chosen tokens' log-probs and probs ---
with torch.no_grad():
    before = []
    for t in range(T):
        logits_t = model(I[t])          # (vocab,)
        logp_t  = log_softmax(logits_t) # (vocab,)
        tok = chosen_tokens[t]
        before.append((tok, float(logp_t[tok]), float(logp_t.exp()[tok])))
before

[(2, -1.750232458114624, 0.17373354732990265),
 (1, -1.5883560180664062, 0.20426113903522491),
 (4, -1.6373724937438965, 0.19449038803577423)]

In [33]:
# --- Compute REINFORCE loss:  L = -A * sum_t log pi(a_t | s_t) ---
total_logprob = 0.0
for t in range(T):
    # sumamos todos los logprobs de los tokens elegidos
    logits_t = model(I[t])          # (vocab,)
    logp_t  = log_softmax(logits_t) # (vocab,)
    tok = chosen_tokens[t]
    total_logprob = total_logprob + logp_t[tok]
    print(f't={t}: token {tok} logprob={logp_t[tok]} total_logprob={total_logprob}')
print(f'-advantage={-advantage}\n-advantage * total_logprob = {-advantage} - {total_logprob}')
loss = - advantage * total_logprob
loss

t=0: token 2 logprob=-1.750232458114624 total_logprob=-1.750232458114624
t=1: token 1 logprob=-1.5883560180664062 total_logprob=-3.3385884761810303
t=2: token 4 logprob=-1.6373724937438965 total_logprob=-4.975960731506348
-advantage=-0.4
-advantage * total_logprob = -0.4 - -4.975960731506348


tensor(1.9904, grad_fn=<MulBackward0>)

In [34]:
# Backprop + step
opt.zero_grad()
loss.backward()
opt.step()

model.weight

Parameter containing:
tensor([[ 0.0207,  0.0412, -0.0441],
        [ 0.0472,  0.2640, -0.0441],
        [ 0.0930,  0.2376, -0.0941],
        [-0.0219, -0.0392,  0.0631],
        [ 0.1794, -0.1253,  0.1386]], requires_grad=True)

In [37]:
# --- Forward AFTER update: record chosen tokens' log-probs and probs ---
with torch.no_grad():
    after = []
    for t in range(T):
        logits_t = model(I[t])          # (vocab,)
        logp_t  = log_softmax(logits_t) # (vocab,)
        tok = chosen_tokens[t]
        after.append((tok, float(logp_t[tok]), float(logp_t.exp()[tok])))

print("Per-step chosen token probabilities (before -> after):")
for t, ((tok_b, logpb, pb), (tok_a, logpa, pa)) in enumerate(zip(before, after)):
    assert tok_b == tok_a
    arrow = "↑" if pa > pb else ("↓" if pa < pb else "→")
    print(f"  t={t}: token={tok_b}   P_before={pb:.4f} -> P_after={pa:.4f}   ({arrow})")

Per-step chosen token probabilities (before -> after):
  t=0: token=2   P_before=0.1737 -> P_after=0.2055   (↑)
  t=1: token=1   P_before=0.2043 -> P_after=0.2386   (↑)
  t=2: token=4   P_before=0.1945 -> P_after=0.2280   (↑)


## PPO

Limita cuanto se puede cambiar la distribucion para que no hayan pasos demasiado grandes y colapse.

Hay un ratio por token

$$r_t = \frac{\pi_{\text{nueva}}(a_t \mid s_t)}{\pi_{\text{vieja}}(a_t \mid s_t)}$$

Que te dice cuanto cambiaste la prob para un token. Si es 1.3, subiste 30% la probabilidad para ese token. Si es 0.7 bajaste 30% la prob para ese token.

- **Clipping**: Si el paso te lleva a un r_t demasiado grande > 1 + eps, entonces recorto el beneficio
- **KL divergence a un modelo referencia**: Penaliza alejarte del modelo base de referencia (el original) en promedio.
- **Ventaja por paso** (opcional): en vez de tener un unico A global, lo hacemos por paso, con V(s_t) para asignar mejor el credito y premiar a los tokens clave. Si no esta, se puede usar como baseline la media del batch.
- **Entropia**: bonus que mantiene la exploracion y evita colapso de entropia (que se vuelva modo unico)

**PPO Loss**:

$$L = \mathbb{E}\left[ \min\left(r_t A_t, \; \text{clip}(r_t, 1-\epsilon, 1+\epsilon) A_t \right) \right] - \beta \, \mathrm{KL}(\pi_{\text{new}} \| \pi_{\text{ref}}) + \alpha \, \text{Entropía}$$

Tres grandes partes:

- Politica (clipped surrogate objective) -> $\mathbb{E}\left[ \min\left(r_t A_t, \; \text{clip}(r_t, 1-\epsilon, 1+\epsilon) A_t \right) \right]$
- KL -> $- \beta \, \mathrm{KL}(\pi_{\text{new}} \| \pi_{\text{ref}})$
- Entropia -> $\alpha \, \text{Entropía}$


**Gradiente**:

- Lo mismo que REINFORCE, pensamos: Queremos minimizar esta loss. Como hacemos? Que depende de nosotros? los theta. Y donde estan, en pi_new. Todo lo demas es constante porque no depende de nosotros.
- Donde esta pi_new? En el ratio: 
$$r_t = \frac{\pi_{\text{new}}(a_t \mid s_t)}{\pi_{\text{old}}(a_t \mid s_t)} = \exp\left( \log \pi_{\text{new}} - \log \pi_{\text{old}} \right)$$
- Por eso al derivar, todo fluye por pi_new.
- PROPIEDAD por la exp: la derivada del ratio es el propio ratio multiplicado por la derivada del log-prob nuevo:

$$\nabla_\theta r_t = r_t \cdot \nabla_\theta \log \pi_{\text{new}}(a_t \mid s_t)$$
 
$$\nabla_\theta (\exp\left( \log \pi_{\text{new}} - \log \pi_{\text{old}} \right)) = (\exp\left( \log \pi_{\text{new}} - \log \pi_{\text{old}} \right)) \cdot \nabla_\theta \log \pi_{\text{new}}$$

Esto se da porque cuando tenemos una funcion exp:

$y = \exp(g(\theta))$

Al derivar exp, te devuelve exp otra vez, y despues multiplicas por la derivada de lo de adentro (**chain rule**):

$\nabla_\theta y = y \cdot \nabla_\theta g(\theta)$

- pi_old no esta porque no depende de theta, es constante -> $\exp(\log \pi_{\text{new}} - \text constante) $
- Entonces, cuando r_t = 1 (al principio cuando las dos policies son iguales), entonces el gradiente es igual a REINFORCE!

**Politica (clipped surrogate objective)**:

En REINFORCE tenemos A_t que es un empuje al gradiente: $A_t \cdot \nabla_\theta \pi$:
- Grad de log pi: es la flecha (vector de cambios) que te dice como mover los pesos para aumentar el valor de la funcion.
- A: Empuja a favor de subir o bajar la prob, segun el reward. Le puede cambiar el signo (si fue malo) y cambiarle intensidad.
    - Si A > 0: queremos subir la prob
    - Si A < 0: queremos bajar la prob

**Ratio pi_new/pi_old: Importance sampling**

Por que multiplicamos por el ratio? IMPORTANCE SAMPLING (es una correccion estadistica).

Las trayectorias se recolectaron con pi_old (porque es caro volver a samplear), pero querés optimizar como si vinieran de pi_new. Estamos trabajando con samples **out of distribution (off-policy)** -> Si bien PPO medio que se clasifica como on-policy en general, esta parte es off-policy.

El verdadero gradiente que quiero estimar:
$$\mathbb{E}_{\tau \sim \pi_{\text{new}}} \left[ \sum_{t} \nabla_\theta \log \pi_{\text{new}}(a_t \mid s_t) A_t \right]$$
Esto es lo ideal. Es la esperanza bajo la POLITICA NUEVA.

Lo que tenemos en la practica:
- Los datos vienen de pi_old. O sea, que si queremos el expectation (dandole mas o menos peso)

Lo que queremos es CORREGIR LA ESPERANZA.
- Un promedio comun es: 1/N sum(x_i) -> cada ejemplo pesa igual. Esto es igual a: sum(x_i * 1/N)
- Un promedio ponderado es: sum(x_i*w_i) / sum(w_i) -> para que sea promedio (suma de pesos = 1). NORMALIZADO.
- En IMPORTANCE SAMPLING queremos el promedio COMO SI viniera de otra distribucion. 
    - Se suele usar version no normalizada por propiedad estadistica: E_q(r(x)*f(x)) = E_p(f(x)) cuando el ratio esta calculado exacto p(x)/q(x)
    - Si tomamos muestras de q, multiplicamos cada valor por r y hacemos el promedio simple 1/N, ya recuperamos la expectativa bajo p: (no hace falta dividir por sum(w_i)). Quedaria: **1/N sum(w_i * x_i)**
    - Por eso en PPO vemos r dentro de la expectation! Es simplemente la ponderacion de cada 

Por que? Porque si hicieramos 1/N, le estamos dando importancias distintas a trayectorias. Le damos la importancia de pi_old, que fue desde donde sampleamos, cuando en realidad le queremos dar la importancia de pi_new. IS es promediar con pesos w en vez de 1/N, para que tus muestras (que vienen de la distribución vieja) imiten el promedio “bajo la nueva”. Listo. Eso es IS.

En PPO:
- Tenemos dos expectations/promedios: outer (entre ejemplos (trayectorias) del minibatch), inner (entre tokens dentro de trayectoria). Este ultimo a veces se lo pone como suma nada mas.
- Lo ideal seria ponderar por trayectoria completa, o sea en el outer. El tema es que no queda muy bien porque habria que hacer el producto de los ratio...
- Se suele **ponderar por token** para aproximar a esa ponderacion por trayectoria. Por eso es que vemos el ratio pi_new/pi_old dentro del inner expectation.

$$
\mathbb{E}_{\tau \sim \pi_{\text{old}}} \left[ \mathbb{E}_{\text{tokens} \in \tau} \left[
\frac{\pi_{\text{new}}(a \mid s)}{\pi_{\text{old}}(a \mid s)} \nabla_\theta \log \pi_{\text{new}}(a \mid s) A
\right] \right]
$$

O solo con suma, como se suele hacer a veces:

$$
\mathbb{E}_{\tau \sim \pi_{\text{old}}} \left[ \sum_{(s,a) \in \tau} \frac{\pi_{\text{new}}(a \mid s)}{\pi_{\text{old}}(a \mid s)} \nabla_\theta \log \pi_{\text{new}}(a \mid s) A \right]
$$

Efectos:
- A la vez de empujarlo con A_t, tambien lo empujamos por el ratio.
- Esto es porque vimos que el grad queda: $\nabla_\theta r_t = r_t \cdot \nabla_\theta \log \pi_{\text{new}}$
- Entonces, los empujes PPO quedarian: $A_t \cdot r_t \cdot \nabla_\theta \log \pi_{\text{new}}$
- Entonces, APARTE DEL **A_t**, ahora tambien lo escalamos por **r_t**: 
    - Si r_t == 1: queda igual a REINFORCE
    - Si r_t > 1: la pi_new ya hace mas probable esa accion que pi_old.
    - Si r_t < 1: la pi_new ya hace menos probable esa accion que pi_old.
- EFECTOS DE ESCALAMIENTO DE GRADIENTES (A_t): 
    - Si A > 0, queremos aumentar la prob de esa accion.
    - Si A < 0, queremos bajar la prob de esa accion.



**CLIP en PPO**:
Importance sampling puede dar pesos extremos (alta varianza). PPO mantiene la corrección pero pone guardarraíles: si r_t se va fuera de [1-ϵ, 1+ϵ] en la dirección que “conviene” a A_t, corta el empuje (grad = 0 para ese token en ese minibatch).


---

**FLUJO**

Los policies new NO SAMPLEAN. Lo unico que samplea es el OLD (se va actualizando cada tanto).

- Tenemos un modelo pi_old. Tenemos prompts. Creamos minibatches (por ejemplo 4 prompts en cada minibatch) y tenemos 3 minibatches (12 ejemplos en total). 
- Sampleamos la respuesta usando pi_old y guardamos toda la info (probs de los tokens para CADA step, reward, etc) para cada prompt en todos los minibatches.
- Ahora empieza la primera corrida de varios EPOCH (1 epoch = 1 pasada por todos los minibatches). Esto se va a hacer SIN HACER NINGUN SAMPLEO A MODELOS. Solo con los datos que estan.
- Agarramos el primer minibatch de 4 ejemplos. Para cada ejemplo ya tenemos la secuencia/respuesta originada por el pi_old. Agarramos el primer ejemplo y vamos token por token de la secuencia guardad por el pi_old y vamos comparando los tokens seleccionados con la prob que arroja el modelo (o sea vamos usando el modelo pi_new para ver la prob que arroja para los tokens muestrados originalmente por el pi_old). Obviamente en la primera vuelta va a ser igual o casi igual que el pi_old porque es el mismo modelo, todavia no se actualizo. Usamos la formula de PPO, sumamos para cada token en la secuencia. -> **El advantage despues lo veo bien**
- Lo mismo para cada ejemplo del batch. Y promediamos para cada ejemplo del batch.
- Despues de cada minibatch, backprop y optimizamos!
- Seguimos con los minibatches hasta terminar todos, hasta ahi 1 epoch
- Hacemos los varios epochs lo mismo. OJO, el pi_new va acumulando los cambios PERO NO VA SAMPLEANDO.
- DESPUES DE ESO, volvemos a samplear pero con el pi_new que reemplaza al pi_old.

Si bien no sampleamos nuevas secuencias con los pi_new a medida que van cambiando, los vamos usando activamente para calcular la prob de cada token de la secuencia original no? O sea tienen un uso muy alto. 
A su vez,es como que los cambios en pi_new se van acumulando durante cada minibatch y eso por varios epochs no? O sea, acumulamos cambios y despues de varios epochs ahi lo usamos para generar una respuesta nueva?


---

**Advantage**

Se hace con actor-critic.

aunque en un episodio ves un solo reward, el crítico se entrena en muchos episodios distintos, y aprende a estimar cuánto se espera desde cada paso en promedio. Esa expectativa por estado es lo que te permite calcular ventajas por token.

El crítico trata de aprender una expectativa por estado, no un número fijo por episodio.
- En el estado justo antes del último token, el crítico aprende a decir: “si sigo desde acá, en promedio espero 0.7 de reward”.
- En un estado más temprano (cuando recién arrancaste a escribir), el crítico podría decir: “desde acá, en promedio espero 0.2 de reward, porque todavía es muy fácil equivocarse”.
- O sea: el crítico no ve solo “un reward concreto”, sino que aprende el promedio esperado desde cada posición a lo largo de muchos episodios distintos. 
- Se entrena en paralelo con su propio value loss.

Como evitar que de mucho advantage a los primeros tokens:
1. Descuento (\gamma<1)
   El crédito que llega desde el final se atenúa hacia atrás: cuanto más lejos esté el token del reward, menos recibe. (Si (\gamma=0.99), a 50 pasos ya cayó mucho).

2. GAE ((\lambda))
   No propagás todo el retorno crudo: usás “sorpresas” (\delta_t) y las filtrás con (\gamma\lambda).
    ⇒ Los primeros tokens reciben poquito y “suavizado”.

3. El crítico aprende “lo esperable” por estado
   Al principio, sí, (V(s)) temprano suele ser bajo. Pero tras ver muchos episodios buenos, el crítico sube su (V(s)) en estados iniciales.
   ⇒ El *advantage* temprano deja de ser grande porque realidad – expectativa se achica.

4. Normalización del advantage
   En práctica se hace A ← (A − mean)/std por minibatch
   ⇒ Evita outliers que dominen el update (ningún tramo de la secuencia “se roba” todo el gradiente).

5. Más “frenos” del policy update

   * Clip PPO corta empujes grandes token-a-token si el ratio (r_t) se va.
   * KL a modelo de referencia otro freno global.
   * Máscaras y pesos podés no dar crédito al *prompt* o bajar el peso a los primeros tokens.



**Ejemplo Importance sampling**

Simple (por trayectoria o outer loop)

In [32]:
import numpy as np
import pandas as pd 

# Distribución objetivo p y de muestreo q
p = {"A": 0.2, "B": 0.8}   # lo que queremos
q = {"A": 0.6, "B": 0.4}   # de donde muestreamos

# f(x) = "gradiente" de cada caso
f = {"A": 10, "B": 0}

# Valor real esperado bajo p
true_expectation = sum(p[x] * f[x] for x in p)
print("Valor real (E_p[f]):", true_expectation)

Valor real (E_p[f]): 2.0


In [33]:
# Muestreamos desde q
N = 1000
samples = np.random.choice(list(q.keys()), size=N, p=list(q.values()))
print(f"Muestras de q:\n{pd.Series(samples).value_counts()}")

Muestras de q:
A    620
B    380
Name: count, dtype: int64


In [34]:
# Promedio ingenuo
naive_estimate = np.mean([f[x] for x in samples])
print("Promedio ingenuo (E_q):", naive_estimate)

Promedio ingenuo (E_q): 6.2


In [35]:
# Importance Sampling (peso w = p/q)
weights = [p[x]/q[x] for x in samples]
# print(f'weights (ratios: p(x)/q(x)): {weights}')
is_estimate = np.mean([w * f[x] for w, x in zip(weights, samples)])
print("Estimador IS (∑ w(x) f(x) / N):", round(is_estimate, 3))

Estimador IS (∑ w(x) f(x) / N): 2.067


Ejemplo mostrando **diferentes tipos de Important sampling**:
- objetivo / ground truth
- IS "bien" por trayectoria
- IS "aprox" por token

In [1]:
import numpy as np
rng = np.random.default_rng(0)  # cambiá la seed si querés

# Largo de secuencia (tokens por trayectoria)
T = 10

ACTIONS = ["A", "B"]  # mantenemos 2 acciones para simpleza

# Políticas por paso (pi_old y pi_new). Cada paso t tiene probs para A/B.
# Podés editarlas a gusto (asegurate que sumen 1 en cada paso).
pi_old = [{"A": 0.8, "B": 0.2} if t % 2 == 0 else {"A": 0.3, "B": 0.7} for t in range(T)]
pi_new = [{"A": 0.5, "B": 0.5} if t % 2 == 0 else {"A": 0.6, "B": 0.4} for t in range(T)]

# Advantages por token (pueden ser constantes o por paso)
A_t = [ +1.0 if t % 2 == 0 else -0.5 for t in range(T) ]  # ej: alterna +1 y -0.5

def ratio_token(t, a):  # r_t = pi_new/pi_old
    return pi_new[t][a] / pi_old[t][a]
pi_old

[{'A': 0.8, 'B': 0.2},
 {'A': 0.3, 'B': 0.7},
 {'A': 0.8, 'B': 0.2},
 {'A': 0.3, 'B': 0.7},
 {'A': 0.8, 'B': 0.2},
 {'A': 0.3, 'B': 0.7},
 {'A': 0.8, 'B': 0.2},
 {'A': 0.3, 'B': 0.7},
 {'A': 0.8, 'B': 0.2},
 {'A': 0.3, 'B': 0.7}]

In [2]:
def seq_prob(policy, seq):
    p = 1.0
    for t, a in enumerate(seq):
        p *= policy[t][a]
    return p

def ground_truth_under_new():
    # E_{tau ~ pi_new}[sum_t A_t]
    # (acá el integrando no depende de la secuencia, así que es la suma A_1+A_2)
    return sum(A_t)

GT = ground_truth_under_new()
print("Ground truth E_{pi_new}[sum A_t] =", GT)

Ground truth E_{pi_new}[sum A_t] = 2.5


In [3]:
N = 16  # cambiá N libremente

def sample_traj_from_old():
    seq = []
    p_old = 1.0
    p_new = 1.0
    for t in range(T):
        a = rng.choice(ACTIONS, p=[pi_old[t]["A"], pi_old[t]["B"]])
        seq.append(a)
        p_old *= pi_old[t][a]
        p_new *= pi_new[t][a]
    return seq, p_old, p_new

batch = [sample_traj_from_old() for _ in range(N)]

print("Primeras 3 trayectorias:")
for i, (seq, po, pn) in enumerate(batch[:3]):
    print(f"{i}: {seq} | p_old={po:.6f} | p_new={pn:.6f}")

Primeras 3 trayectorias:
0: [np.str_('A'), np.str_('A'), np.str_('A'), np.str_('A'), np.str_('B'), np.str_('B'), np.str_('A'), np.str_('B'), np.str_('A'), np.str_('B')] | p_old=0.002529 | p_new=0.000720
1: [np.str_('B'), np.str_('A'), np.str_('B'), np.str_('A'), np.str_('A'), np.str_('A'), np.str_('B'), np.str_('B'), np.str_('A'), np.str_('B')] | p_old=0.000068 | p_new=0.001080
2: [np.str_('A'), np.str_('A'), np.str_('A'), np.str_('B'), np.str_('A'), np.str_('B'), np.str_('B'), np.str_('B'), np.str_('A'), np.str_('B')] | p_old=0.005901 | p_new=0.000480


In [4]:
# Exact IS (traj-level): W_traj = p_new(seq)/p_old(seq)
terms_traj = []
for (seq, p_old, p_new) in batch:
    W_traj = p_new / p_old                 # producto de ratios por token
    inner = sum(A_t)                       # sum_t A_t  (tratamos ∇log como 1 para ver sólo pesos)
    terms_traj.append(W_traj * inner)

est_exact_traj = np.mean(terms_traj)
print("Exact IS (traj-level):", est_exact_traj)


Exact IS (traj-level): 6.23579417360512


In [5]:
terms_token = []
for (seq, p_old, p_new) in batch:
    inner = 0.0
    for t, a in enumerate(seq):
        inner += ratio_token(t, a) * A_t[t]   # r_t * A_t  (∇log ~ 1 para ilustrar)
    terms_token.append(inner)

est_surrogate = np.mean(terms_token)
print("Surrogate (token-level):", est_surrogate)

Surrogate (token-level): 3.247767857142857


In [ ]:
import numpy as np
import pandas as pd
from itertools import product

rng = np.random.default_rng(123)

# === Controls you can edit ===
T_list  = [3, 10, 25, 100]   # sequence lengths to try
N_list  = [3, 10, 25, 100]   # batch sizes to try
reps    = 200                # trials per (T, N)

# Define 2-action policies that alternate structure across time
def make_policies(T):
    # old policy alternates (A-heavy at even t, B-heavy at odd t)
    pi_old = [{"A": 0.8, "B": 0.2} if (t % 2 == 0) else {"A": 0.3, "B": 0.7} for t in range(T)]
    # new policy is more balanced / slightly A-heavy on odd t
    pi_new = [{"A": 0.5, "B": 0.5} if (t % 2 == 0) else {"A": 0.6, "B": 0.4} for t in range(T)]
    return pi_old, pi_new

# Advantages per token (can be any shape you want)
def make_advantages(T):
    # Alternate +1 and -0.5 by time step
    return [1.0 if (t % 2 == 0) else -0.5 for t in range(T)]

ACTIONS = ["A", "B"]

def sample_traj_from_old(pi_old, pi_new):
    seq = []
    p_old = 1.0
    p_new = 1.0
    for t in range(len(pi_old)):
        a = rng.choice(ACTIONS, p=[pi_old[t]["A"], pi_old[t]["B"]])
        seq.append(a)
        p_old *= pi_old[t][a]
        p_new *= pi_new[t][a]
    return seq, p_old, p_new

def ratio_token(pi_old, pi_new, t, a):
    return pi_new[t][a] / pi_old[t][a]

rows = []
for T, N in product(T_list, N_list):
    pi_old, pi_new = make_policies(T)
    A_t = make_advantages(T)

    # Ground truth bajo pi_new: como A_t no depende de la acción, es sum(A_t)
    GT = float(sum(A_t))

    exact_vals = []
    sur_vals   = []
    for _ in range(reps):
        # build one batch
        batch = [sample_traj_from_old(pi_old, pi_new) for _ in range(N)]

        # Exact IS (trajectory-level): weight = product of ratios (p_new/p_old)
        terms_traj = []
        for (seq, p_old, p_new) in batch:
            W_traj = p_new / p_old
            # inner sum over tokens (∇log term ~ 1 solo para comparar pesos)
            inner = sum(A_t)
            terms_traj.append(W_traj * inner)
        exact_vals.append(np.mean(terms_traj))

        # Surrogate (token-level): sum of r_t * A_t; then average over trajectories
        terms_token = []
        for (seq, p_old, p_new) in batch:
            inner = 0.0
            for t, a in enumerate(seq):
                inner += ratio_token(pi_old, pi_new, t, a) * A_t[t]
            terms_token.append(inner)
        sur_vals.append(np.mean(terms_token))

    rows.append({
        "T": T,
        "N": N,
        "GT": GT,
        "Exact_mean": float(np.mean(exact_vals)),
        "Exact_std": float(np.std(exact_vals, ddof=1)),
        "Sur_mean": float(np.mean(sur_vals)),
        "Sur_std": float(np.std(sur_vals, ddof=1)),
        "Exact_bias": float(np.mean(exact_vals) - GT),
        "Sur_bias": float(np.mean(sur_vals) - GT),
    })

df = pd.DataFrame(rows).sort_values(["T","N"]).reset_index(drop=True)
print("Exploración: Exact IS (traj) vs Surrogate (token) según T y N")
df

Exploración: Exact IS (traj) vs Surrogate (token) según T y N
  T   N   GT  Exact_mean  Exact_std  Sur_mean  Sur_std  Exact_bias  Sur_bias
  3   3  1.5    1.515346   1.503303  1.462351 0.602518    0.015346 -0.037649
  3  10  1.5    1.376032   0.624431  1.485491 0.338125   -0.123968 -0.014509
  3  25  1.5    1.478170   0.490886  1.478482 0.212735   -0.021830 -0.021518
  3 100  1.5    1.462910   0.215665  1.496237 0.108755   -0.037090 -0.003763
 10   3  2.5    3.295577  13.788061  2.750149 1.063058    0.795577  0.250149
 10  10  2.5    1.963271   2.152243  2.468437 0.549195   -0.536729 -0.031563
 10  25  2.5    2.245605   2.275499  2.545161 0.359974   -0.254395  0.045161
 10 100  2.5    2.451338   2.042685  2.482991 0.179396   -0.048662 -0.017009
 25   3  7.0    2.679560  10.801364  6.850298 1.713741   -4.320440 -0.149702
 25  10  7.0    5.779369  40.098797  6.920982 0.849592   -1.220631 -0.079018
 25  25  7.0    4.489134  19.168396  6.966018 0.578169   -2.510866 -0.033982
 25 100  7.0  

- Exact trajectory IS = teórico, sin sesgo, pero inviable por varianza (crece mucho con T). Al multiplicar muchos ratios, la **varianza explota** porque hay sub/overflow y el cociente pierde precision.
- Token surrogate = práctico, sesgado, pero con varianza baja y entrenable.

Es el típico trade-off de PPO: preferimos estabilidad aunque perdamos exactitud.

**PPO simplificado** (solo parte de policy clipped. Sin KL ni entropy)

In [56]:
import torch
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(0)

# Hiperparámetros chiquitos
vocab_size = 5
T = 3
lr = 0.5
eps_clip = 0.2   # zona segura [0.8, 1.2]
advantage = 0.4  # mismo A para todos los pasos (reward final - baseline)

# Modelo lineal sin bias (como el tuyo)
model = nn.Linear(T, vocab_size, bias=False)
with torch.no_grad():
    model.weight.copy_(0.1 * torch.randn(vocab_size, T))

opt = optim.SGD(model.parameters(), lr=lr)
log_softmax = nn.LogSoftmax(dim=-1)

# Estados one-hot (t = 0..T-1)
I = torch.eye(T)

# Elegimos una secuencia de tokens (simula lo que generó π_old)
chosen_tokens = [2, 1, 4]

In [57]:
# ----------------------------------------------------------
# 1) "Foto" de π_old: guardamos logprob_old en esos (s_t, a_t)
# ----------------------------------------------------------
with torch.no_grad():
    logprob_old = []
    for t in range(T):
        logits_t = model(I[t])          # (vocab,)
        logp_t  = log_softmax(logits_t) # (vocab,)
        tok = chosen_tokens[t]
        logprob_old.append(float(logp_t[tok]))
    logprob_old = torch.tensor(logprob_old, dtype=torch.float32)

print("logprob_old por paso:", [f"{x:.4f}" for x in logprob_old.tolist()])

logprob_old por paso: ['-1.7502', '-1.5884', '-1.6374']


In [58]:
# ----------------------------------------------------------
# 2) BEFORE update: logprob_new y ratio (r_t ≈ 1 al inicio)
# ----------------------------------------------------------
with torch.no_grad():
    before = []
    ratio_before = []
    for t in range(T):
        logits_t = model(I[t])
        logp_t  = log_softmax(logits_t)
        tok = chosen_tokens[t]
        logp_new = logp_t[tok]
        r_t = torch.exp(logp_new - logprob_old[t])
        before.append((tok, float(logp_new), float(logp_t.exp()[tok])))
        ratio_before.append(float(r_t))
print("logprob_new BEFORE:", [f"{x[1]:.4f}" for x in before])
print("P_new BEFORE      :", [f"{x[2]:.4f}" for x in before])
print("r_t BEFORE        :", [f"{r:.4f}" for r in ratio_before])

logprob_new BEFORE: ['-1.7502', '-1.5884', '-1.6374']
P_new BEFORE      : ['0.1737', '0.2043', '0.1945']
r_t BEFORE        : ['1.0000', '1.0000', '1.0000']


In [59]:
# ----------------------------------------------------------
# 3) Loss PPO (sólo política, clipeada)
#    L = - mean( min( r_t*A, clamp(r_t)*A ) )
# ----------------------------------------------------------
advantages = torch.full((T,), fill_value=advantage, dtype=torch.float32)

total_unclipped = 0.0
total_clipped = 0.0
for t in range(T):
    logits_t = model(I[t])
    logp_t  = log_softmax(logits_t)
    tok = chosen_tokens[t]
    logp_new = logp_t[tok]
    r_t = torch.exp(logp_new - logprob_old[t])
    unclipped = r_t * advantages[t]
    clipped_r = torch.clamp(r_t, 1.0 - eps_clip, 1.0 + eps_clip)
    clipped = clipped_r * advantages[t]
    total_unclipped = total_unclipped + unclipped
    total_clipped   = total_clipped   + clipped
    print(f"t={t}: tok={tok}  logp_old={logprob_old[t]:+.4f}  logp_new={logp_new:+.4f}  r_t={float(r_t):.4f}  "
          f"uncl={float(unclipped):+.4f}  clp={float(clipped):+.4f}  picked=min(uncl,clp)")


t=0: tok=2  logp_old=-1.7502  logp_new=-1.7502  r_t=1.0000  uncl=+0.4000  clp=+0.4000  picked=min(uncl,clp)
t=1: tok=1  logp_old=-1.5884  logp_new=-1.5884  r_t=1.0000  uncl=+0.4000  clp=+0.4000  picked=min(uncl,clp)
t=2: tok=4  logp_old=-1.6374  logp_new=-1.6374  r_t=1.0000  uncl=+0.4000  clp=+0.4000  picked=min(uncl,clp)


In [60]:
# Promediamos el surrogate (min) y lo negamos para minimizar
surrogate = torch.minimum(total_unclipped, total_clipped) / T
loss = -surrogate
print(f"\nSurrogate (mean): {float(surrogate):+.6f}  ->  PPO policy loss = {float(loss):+.6f}")



Surrogate (mean): +0.400000  ->  PPO policy loss = -0.400000


In [61]:
# Backprop + step
opt.zero_grad()
loss.backward()
opt.step()


In [62]:
# ----------------------------------------------------------
# 4) AFTER update: ver cómo cambiaron P_new de los tokens elegidos
# ----------------------------------------------------------
with torch.no_grad():
    after = []
    ratio_after = []
    for t in range(T):
        logits_t = model(I[t])
        logp_t  = log_softmax(logits_t)
        tok = chosen_tokens[t]
        logp_new = logp_t[tok]
        r_t = torch.exp(logp_new - logprob_old[t])
        after.append((tok, float(logp_new), float(logp_t.exp()[tok])))
        ratio_after.append(float(r_t))

print("\nPer-step chosen token probabilities (before -> after):")
for t, ((tok_b, logpb, pb), (tok_a, logpa, pa)) in enumerate(zip(before, after)):
    assert tok_b == tok_a
    arrow = "↑" if pa > pb else ("↓" if pa < pb else "→")
    print(f"  t={t}: token={tok_b}   P_before={pb:.4f} -> P_after={pa:.4f}   ({arrow})")

print("r_t AFTER         :", [f"{r:.4f}" for r in ratio_after])



Per-step chosen token probabilities (before -> after):
  t=0: token=2   P_before=0.1737 -> P_after=0.1839   (↑)
  t=1: token=1   P_before=0.2043 -> P_after=0.2153   (↑)
  t=2: token=4   P_before=0.1945 -> P_after=0.2052   (↑)
r_t AFTER         : ['1.0583', '1.0540', '1.0552']


## GRPO

Es similar a PPO pero cambia en el **calculo del Advantage**.

Cantidad de trayectorias originales por prompt:
- En **PPO** teniamos varios prompts y una sola respuesta original por prompt. Luego ibamos modificando pi_new prediciendo sobre esa misma trayectoria generada original (una por cada prompt). 
- Ahora en **GRPO** tenemos **k respuestas por prompt**. Tambien vamos modificando pi_new prediciendo sobre las trayectorias generadas originales (k por prompt).

Minibatches:
- En PPO, si en el batch total tenemos 256 prompts, generamos 256 trayectorias y, si el batch_size=16, nos quedaban 16 mini batches
- En GRPO, si en el batch total tenemos 256 prompts, generamos 256*k trayectorias (si k=4 -> 1024 trayectorias). Y si batch_size=16, nos quedan 64 mini batches. O sea, en un minibatch pueden haber dos o mas casos del mismo prompt pero con distintas trayectorias originales.

**Baseline y Advantage**:
- En PPO se calculaba con un critic o value function. Dos formas: TD advantage por paso o GAE (lo clasico). Involucraba otro modelo aprendible lo cual no era bueno.
- En GRPO, tenemos 256 prompts. Para cada prompt generamos k=4 trayectorias -> eso es un GRUPO. Y esas generan rewards. Tomamos la media y desvio de las rewards del grupo y ese es el **baseline**.
    - Lo malo es que ahora no tenemos Advantage por token, solo advantage por trayectoria.

$$A_i = r_i - \bar{r}_{\text{prompt}} \text{ ------ (y opcionalmente} A_i \leftarrow \frac{A_i}{\mathrm{std}}\text{)}$$  

**Objetivo**
